# Voronoi Slice Plots
"Playground" notebook used for generating isolated voronoi slice plots.

In [ ]:
# Loading libraries and key coordinates
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import FuncFormatter
from scipy import interpolate
from matplotlib.patches import Arc
from matplotlib.patches import Circle
import seaborn as sns

from utilities import Temp_S

################################################
##### PHYSICAL CONSTANTS #####
HYDROGEN_MASS_FRACTION = 0.76
PROTON_MASS_GRAMS = 1.67262192e-24 # mass of proton in grams
gamma = 5/3
kb = 1.3807e-16 # Boltzmann Constant in CGS or erg/K
GRAVITIONAL_CONSTANT_IN_CGS = 6.6738e-8
HUBBLE = 3.2407789e-18
M_PI = 3.14159265358979323846
seconds_in_myrs = 3.15576e+13
seconds_in_yrs = 3.154e+7
sb_constant_cgs = 5.670374419e-5 # ergs/cm^2/K^4/s
T_COLD_MAX = 3e4
n_bins = 300

##### UNITS #####
UnitVelocity_in_cm_per_s = 1e5 # 10 km/sec 
UnitLength_in_cm = 3.085678e21 # 1 kpc
UnitMass_in_g  = 1.989e33 # 1 solar mass
UnitTime_in_s = UnitLength_in_cm / UnitVelocity_in_cm_per_s # 3.08568e+16 seconds 
UnitEnergy_in_cgs = UnitMass_in_g * pow(UnitLength_in_cm, 2) / pow(UnitTime_in_s, 2) # 1.9889999999999999e+43 erg
UnitDensity_in_cgs = UnitMass_in_g / pow(UnitLength_in_cm, 3) # 6.76989801444063e-32 g/cm^3
UnitPressure_in_cgs = UnitMass_in_g / UnitLength_in_cm / pow(UnitTime_in_s, 2) # 6.769911178294542e-22 barye
UnitNumberDensity = UnitDensity_in_cgs/PROTON_MASS_GRAMS
UnitEnergyDensity = UnitEnergy_in_cgs/pow(UnitLength_in_cm, 3)

G_code = GRAVITIONAL_CONSTANT_IN_CGS/(pow(UnitLength_in_cm,3) * pow(UnitMass_in_g, -1) * pow(UnitTime_in_s, -2)) 
Hubble_code = HUBBLE * UnitTime_in_s # All.Hubble in Arepo

def make_voronoi_slice_edge(gas_xyz, gas_values, image_num_pixels, image_y_value, image_xz_max): 
    interp = interpolate.NearestNDInterpolator(gas_xyz, gas_values)  # declare an interpolator of coordinates and values associated with the coordinates
    s = image_xz_max/image_num_pixels
    xs = np.arange(np.min(gas_xyz), np.max(gas_xyz)+s, s)
    zs = np.arange(np.min(gas_xyz), np.max(gas_xyz)+s, s)

    X,Z = np.meshgrid(xs,zs)
    M_coords = np.transpose(np.vstack([X.ravel(), np.full(len(X.ravel()), image_y_value), Z.ravel()]))
    result = np.transpose(interp(M_coords).reshape(len(xs), len(zs)))

    return result, np.array(xs), np.array(zs)

def make_voronoi_slice_face(gas_xyz, gas_values, image_num_pixels, image_z_value, image_xy_max): 
    interp = interpolate.NearestNDInterpolator(gas_xyz, gas_values)
    s = image_xy_max/image_num_pixels
    xs = np.arange(np.min(gas_xyz), np.max(gas_xyz)+s, s)
    ys = np.arange(np.min(gas_xyz), np.max(gas_xyz)+s, s) 

    X,Y = np.meshgrid(xs,ys)
    M_coords = np.transpose(np.vstack([X.ravel(), Y.ravel(), np.full(len(X.ravel()), image_z_value)] ))
    result = np.transpose(interp(M_coords).reshape(len(ys), len(xs)))

    return result, np.array(xs), np.array(ys)


def plot_face(ax, coordinates, value, bins, center, boxsize, minimum, maximum, log):
    stat, x_edge, y_edge = make_voronoi_slice_face(coordinates, value, bins, center, boxsize)
    ax.set(xlim=(histb_l, histb_h), ylim=(histb_l, histb_h)) 
    face_mesh = ax.pcolormesh(x_edge, y_edge, stat.T, shading='auto')
    if (log): face_mesh.set_norm(colors.LogNorm(vmin=minimum, vmax=maximum))
    else: face_mesh.set_clim(minimum, maximum)
    # custom_tick_labels = lambda x: f"{x - boxsize/2:.0f}"
    ax.xaxis.set_major_formatter(FuncFormatter(custom_tick_labels))
    ax.yaxis.set_major_formatter(FuncFormatter(custom_tick_labels))
    ax.set_xlabel('X [kpc]')
    ax.set_ylabel('Y [kpc]')

def plot_edge(ax, coordinates, value, bins, center, boxsize, minimum, maximum, log):
    stat, x_edge, z_edge = make_voronoi_slice_edge(coordinates, value, bins, center, boxsize)
    ax.set(xlim=(histb_l, histb_h), ylim=(histb_l, histb_h)) 
    edge_mesh = ax.pcolormesh(x_edge, z_edge, stat.T, shading='auto')
    if (log): edge_mesh.set_norm(colors.LogNorm(vmin=minimum, vmax=maximum))
    else: edge_mesh.set_clim(minimum, maximum)
    # custom_tick_labels = lambda x, pos: f"{x - boxsize/2:.0f}"
    ax.xaxis.set_major_formatter(FuncFormatter(custom_tick_labels))
    ax.yaxis.set_major_formatter(FuncFormatter(custom_tick_labels))
    ax.set_xlabel('X [kpc]', fontsize=14)
    ax.set_ylabel('Z [kpc]', fontsize=14)

###### KEY SIMULATION PARAMETERS ######
boxsize = 100
half_boxsize = boxsize/2 
center_boxsize = 10
center_boxsize_large = 15 
cells_per_dim = 301 
cells_per_dim_large = 451 # for the MW plots

plt.style.use("seaborn-paper")

def custom_tick_labels(x, pos):
    return f"{x - boxsize/2:.0f}"

In [ ]:
deviation = 30
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2  + deviation # boundary of histogram - upper bound
angle_l = 60
conical_file = "/cc85_disk/output_center_ref/snap_200.hdf5"
data_conical = {}
with h5py.File(conical_file, 'r') as f:
    for key in f['PartType0']:
        data_conical[key] = f['PartType0'][key][()]
    header_conical = dict(f['Header'].attrs)
    parameters_conical = dict(f['Parameters'].attrs)
R = parameters_conical["injection_radius"]
coord_conical = data_conical["Coordinates"]
x_conical = data_conical["Coordinates"][:,0] 
y_conical = data_conical["Coordinates"][:,1]
z_conical = data_conical["Coordinates"][:,2]
rho_conical = data_conical["Density"]
masses_conical = data_conical["Masses"]
dx = center_boxsize/cells_per_dim
rad_x, rad_y, rad_z = x_conical - 0.5*boxsize, y_conical - 0.5*boxsize, z_conical - 0.5*boxsize 
radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2) 

tan_theta = rad_x/rad_z
theta = np.arccos(np.abs(rad_z)/(radius + dx/1e6))*180/np.pi 
conical_region = (np.abs(theta) <= angle_l) & (np.abs(rad_z) >= 0.3) # Excludes anything with absolute angles greater than 60 
lower_bound, upper_bound = half_boxsize - dx*3, half_boxsize + dx*3
edge_mask = (y_conical >=lower_bound) & (y_conical <= upper_bound) 

fig_bicone = plt.figure(figsize=(6,5))
fig_bicone.set_rasterized(True)
ax_bicone_vor = fig_bicone.add_subplot(111)
plot_edge(ax_bicone_vor, coord_conical[edge_mask], rho_conical[edge_mask]*UnitNumberDensity, cells_per_dim*2, half_boxsize, center_boxsize, 1e-5, 1, log=True)
density_mesh = ax_bicone_vor.collections[0]
# ax_bicone_vor.axhline(half_boxsize - 0.5, color="white", linewidth=0.5)
# ax_bicone_vor.axhline(half_boxsize + 0.5, color="white", linewidth=0.5)
density_mesh.set_cmap("mako")
conal_cbar_vor = plt.colorbar(density_mesh, ax = ax_bicone_vor, pad=0.02)
conal_cbar_vor.set_label('Density [log($cm^{-3}$)]', fontsize=13)
labels = [1e-5*(10**(x)) for x in range(0,6)]

conal_cbar_vor.set_ticks(labels)
conal_cbar_vor.set_ticklabels([round(np.log10(label)) for label in (labels)])
ax_bicone_vor.axvline(half_boxsize,linewidth=1, color="white")
ax_bicone_vor.axline((half_boxsize, half_boxsize), slope=np.tan((90 - angle_l)*np.pi/180), linewidth=1, linestyle="dashed", color="white")
ax_bicone_vor.axline((half_boxsize, half_boxsize), slope=-np.tan((90 - angle_l)*np.pi/180), linewidth=1, linestyle="dashed", color="white")
ax_bicone_vor.set_xlabel("X [kpc]", fontsize=13)
ax_bicone_vor.set_ylabel("Z [kpc]", fontsize=13)

ax_bicone_vor.tick_params(axis='both', which='major', labelsize=12)

arc = Arc((half_boxsize,half_boxsize), width=7, height=7, angle=0, theta1=30, theta2=90, color="white", linestyle="dotted", linewidth=2)
# plt.plot(x1,x2, color="white", linestyle="dotted")
ax_bicone_vor.add_patch(arc)


ax_bicone_vor.text(half_boxsize + 0.5, half_boxsize + 6, "$60^{\circ}$", color="white", fontsize="medium")
circle_bi = Circle((half_boxsize,half_boxsize), radius=20, color="white", linestyle="solid", linewidth=2, fill=False)
ax_bicone_vor.add_patch(circle_bi)


plt.savefig("region_analysis_angle_full.pdf", dpi=150, bbox_inches='tight')
plt.show()


### Initial conditions side by side
Made for the purpose of the half-time presentation, as it would be difficult to present all three 6-panel plots on powerpoint 

In [ ]:
deviation = 5
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2  + deviation # boundary of histogram - upper bound

files = ["outflow_disk/Disk_M82/snap_000.hdf5","./moving_mesh_disk/Disk_M821Gyr/snap_000.hdf5", "./moving_mesh_disk/Disk_M821Gyr/snap_100.hdf5"]
text = ["t = 0 Myr", "t = 1 Gyr", "t = 1.5 Gyr"]
sim = ["Start of Static Mesh", "End of Static Mesh", "End of Moving Mesh"]
dx = center_boxsize/300
eps = dx/1e6
lower_bound, upper_bound = half_boxsize - dx*3, half_boxsize + dx*3
fig = plt.figure(figsize=(15, 4))
fig.set_rasterized(True)
ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)  
ax3 = fig.add_subplot(1, 3, 3)
axs = [ax1, ax2, ax3]
data = {}
for i, file in enumerate(files):
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    number_density = density*UnitNumberDensity
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    t = header["Time"]

    ts = int(round(t*1000))
    ''' Get the radius of the box - will be useful in later plots'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radial_coord = np.sqrt(rad_x**2+rad_y**2)
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)
    radial_velocity = (vel_x*rad_x + vel_y*rad_y)/(radial_coord + eps) 
    v_r_spherical = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps) 
    tvx, tvy = vel_x - radial_velocity*rad_x/(radial_coord + eps) , vel_y - radial_velocity*rad_y/(radial_coord + eps) 
    tan_velocity = np.sqrt(tvx**2 + tvy**2)
    temperatures = Temp_S(1, internal_energy)
    
    face_mask = (z_coord >= lower_bound) & (z_coord <= upper_bound) & (radial_coord <= center_boxsize/2*np.sqrt(2))
    edge_mask = (y_coord >= lower_bound) & (y_coord <= upper_bound) # & (radius <= 10)

    plot_edge(axs[i], coord[edge_mask], number_density[edge_mask], n_bins, half_boxsize, center_boxsize, 1e-4, 10, log=True)
    density_mesh = axs[i].collections[0]
    axs[i].text(0.03, 0.91, text[i], transform=axs[i].transAxes, color="white", fontsize="large")
    axs[i].text(0.03, 0.84, sim[i], transform=axs[i].transAxes, color="white", fontsize="large")
    
    density_mesh.set_cmap("mako")
    cbar = plt.colorbar(density_mesh, ax = axs[i], label='Density [log($cm^{-3}$)]')
    labels = [1e-5*(10**(x)) for x in range(1,8)]
    cbar.set_ticks(labels)
    cbar.set_ticklabels([round(np.log10(label)) for label in (labels)])
plt.savefig("ic_presentation.pdf", dpi=300, bbox_inches='tight', facecolor='white', transparent=False)
plt.show()

### Cooling vs No Cooling Slice

In [ ]:
import matplotlib.patches as patches
plt.style.use("seaborn-paper")

deviation = 30
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2  + deviation # boundary of histogram - upper bound
box_range = 20
z_binning = 100
upper_x = 5
n_bins = 200
dx = center_boxsize/300
lower_bound, upper_bound = boxsize/2 - dx*6, boxsize/2 + dx*6




######### SIMULATION DATA #########
data = {}
files = ["./cc85_disk/output_cooling_params/snap_100.hdf5",  "./outflow_cooling/output_PIE_fid/snap_100.hdf5"]
plt.style.use("seaborn-paper")
fig = plt.figure(figsize=(12,10))
fig.set_rasterized(True)    
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)
angle_l = 60
for i, file in enumerate(files): # select the snapshot range to go through
    with h5py.File(file,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    masses = data["Masses"] 
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    volume = masses/density
    number_density = density*UnitNumberDensity

    if "cc85_disk" in file: temperature = Temp_S(1, internal_energy)
    else: 
        abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)
    t = header["Time"]
    times = t*1000
    ''' Get the radial distance of the box'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)


    edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) # & (radius <= inner_boxsize/2*np.sqrt(3))

    volume = masses/density
    # import pdb; pdb.set_trace()
    #### Velocities - for the center disk plane face####
    radial_velocity_spherical = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps)

    ### PLOTS ###

    if "cc85_disk" in file:
        plot_edge(ax1, coord[edge_mask], temperature[edge_mask], n_bins, half_boxsize, center_boxsize, 1e3, 1e7, log=True)
        T_mesh = ax1.collections[0]
        T_mesh.set_cmap("plasma")
        cbar1 = plt.colorbar(T_mesh, ax = ax1, pad=0.02)
        cbar1.set_label('Temperature [log(K)]', fontsize=14)
        labels = [1*(10**(x)) for x in range(3,8)]
        cbar1.set_ticks(labels)
        cbar1.set_ticklabels([int(np.log10(label)) for label in labels]) 
        background_rect = patches.Rectangle((0, 0.78), width=1, height=0.22, color='black', alpha=0.25, transform=ax1.transAxes, fill=True)
        ax1.add_patch(background_rect)
        ax1.text(0.01, 0.96,"t = %0.3f Myr" % times, transform=ax1.transAxes, color="white", fontname='serif', fontsize=14)
        ax1.text(0.01, 0.91,r'LMC/M82 Disk - No Cooling', transform=ax1.transAxes, color="white", fontname='serif', fontsize=14)
        ax1.text(0.03, 0.86,r"- $\beta = 0.6, \alpha = 0.9$", transform=ax1.transAxes, color="white", fontname='serif',fontsize=14)
        ax1.text(0.03, 0.81,r"- $\dot{M}_{SFR} = 20 M_\odot \, yr^{-1}$, $R_{inject}$ = 300 pc", transform=ax1.transAxes, color="white", fontname='serif', fontsize=14)
        plot_edge(ax3, coord[edge_mask], density[edge_mask]*UnitNumberDensity, n_bins, half_boxsize, center_boxsize, 1e-5, 10, log=True)
        density_mesh = ax3.collections[0]
        density_mesh.set_cmap("mako")
        cbar3 = plt.colorbar(density_mesh, ax = ax3, pad=0.02)
        cbar3.set_label(r'Density [log($\rm cm^{-3}$)]', fontsize=14)

        labels = [1e-5*(10**(x)) for x in range(0,7)]
        cbar3.set_ticks(labels)
        cbar3.set_ticklabels([round(np.log10(label)) for label in (labels)])

    else: 
        plot_edge(ax2, coord[edge_mask], temperature[edge_mask], n_bins, half_boxsize, center_boxsize, 1e3, 1e7, log=True)
        T_mesh = ax2.collections[0]
        T_mesh.set_cmap("plasma")
        cbar2 = plt.colorbar(T_mesh, ax = ax2, pad=0.02)
        cbar2.set_label('Temperature [log(K)]', fontsize=14)
        labels = [1*(10**(x)) for x in range(3,8)]
        cbar2.set_ticks(labels)
        cbar2.set_ticklabels([int(np.log10(label)) for label in labels]) 
        background_rect = patches.Rectangle((0, 0.78), width=1, height=0.22, color='black', alpha=0.25, transform=ax2.transAxes, fill=True)
        ax2.add_patch(background_rect)

        ax2.text(0.01, 0.95,"t = %0.3f Myr" % times, transform=ax2.transAxes, color="white", fontsize=14)
        ax2.text(0.01, 0.90,r'LMC/M82 Disk - PIE+CIE Cooling', transform=ax2.transAxes, color="white", fontsize=14)
        ax2.text(0.03, 0.85,r"- $\beta = 0.6, \alpha = 0.9, Z_{disk}= 4 Z_\odot$", transform=ax2.transAxes, color="white", fontsize=14)
        ax2.text(0.03, 0.80,r"- $\dot{M}_{SFR} = 20 M_\odot \, yr^{-1}$, $R_{inject}$ = 300 pc", transform=ax2.transAxes, color="white", fontsize=14)
        ax2.tick_params(axis='both', labelsize="large")

        # ax2.axvline(half_boxsize,linewidth=1, color="white")
        # ax2.axline((half_boxsize, half_boxsize), slope=np.tan((90 - angle_l)*np.pi/180), linewidth=1, linestyle="dashed", color="white")
        # ax2.axline((half_boxsize, half_boxsize), slope=-np.tan((90 - angle_l)*np.pi/180), linewidth=1, linestyle="dashed", color="white")
        # arc = Arc((half_boxsize,half_boxsize), width=3, height=3, angle=0, theta1=30, theta2=90, color="white", linestyle="dotted", linewidth=2)
        # # plt.plot(x1,x2, color="white", linestyle="dotted")
        # ax2.add_patch(arc)
        
        # ax2.text(half_boxsize + 0.1, half_boxsize + 0.5, "$60^{\circ}$", color="black", fontsize="medium")

        plot_edge(ax4, coord[edge_mask], density[edge_mask]*UnitNumberDensity, n_bins, half_boxsize, center_boxsize, 1e-5, 10, log=True)
        density_mesh = ax4.collections[0]
        density_mesh.set_cmap("mako")
        cbar4 = plt.colorbar(density_mesh, ax = ax4, pad=0.02)
        cbar4.set_label(r'Density [log($\rm cm^{-3}$)]', fontsize=14)
        labels = [1e-5*(10**(x)) for x in range(0,7)]
        cbar4.set_ticks(labels)
        cbar4.set_ticklabels([round(np.log10(label)) for label in (labels)])

ax1.tick_params(axis='both', labelsize=12)
ax2.tick_params(axis='both', labelsize=12)
ax3.tick_params(axis='both', labelsize=12)
ax4.tick_params(axis='both', labelsize=12)

cbar1.ax.tick_params(labelsize=12)
cbar2.ax.tick_params(labelsize=12)
cbar3.ax.tick_params(labelsize=12)
cbar4.ax.tick_params(labelsize=12)

plt.tight_layout(w_pad=0.05, h_pad=0.05)


plt.savefig("temp_density_comp.pdf", dpi=300, bbox_inches='tight', facecolor='white', transparent=False)
 

### Time Evolution of the Disks
Compares the density evolution of the outflows between the three disk models across time

In [ ]:

# Non-Cooling
## Injection Radius
# outputs = ["./cc85_disk/output_center_ref/", "output_r300oc"]
## Disk Modsl
outputs = ["./cc85_disk/output_center_ref/", "./cc85_disk/output_center_ref_large/", "./cc85_disk/output_center_ref_small/"]

deviation = 20
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2  + deviation # boundary of histogram - upper bound

plt.style.use("seaborn-paper")

dx = center_boxsize/cells_per_dim
halfbox_inner = center_boxsize/2 
lower_bound, upper_bound = half_boxsize - dx*5, half_boxsize + dx*5
devx = dx/1e4

labeling = ["M82/LMC", "MW", "SMC"]
###############################################################
######### SIMULATION DATA #########
data = {}
times = np.array([0, 20, 40, 100])
snaps = times*2 # very roughly, I am taking a snapshot every 0.5 Myrs and since I have 100 snapshots, the snapshot with the closest time will be t*2
snaps_alt = times
ticks = [30, 35, 40, 45, 50, 55, 60, 65, 70]
fig, axes = plt.subplots(3, 4, figsize=(18, 14))
fig.set_rasterized(True)
x = 0
n_bins = 300
# cmap = plt.cm.snsmako 
axs = axes.flatten()
for i, output in enumerate(outputs):
    s = snaps_alt if any(o in output for o in ["large", "small"]) else snaps
    for j, snap in enumerate(s):
        filename = output + "snap_%03d.hdf5" % snap
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)      
        R = parameters["injection_radius"]
        M_load = parameters["M_load"]
        E_load = parameters["E_load"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity
        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        t = header["Time"]
        ts = int(round(t*1000))
        ''' Get the radius of the box - will be useful in later plots'''
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
        radial_coord = np.sqrt(rad_x**2+rad_y**2)
        radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y)/(radial_coord + devx) 
        v_r_spherical = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + devx) 
        tvx, tvy = vel_x - radial_velocity*rad_x/(radial_coord + devx) , vel_y - radial_velocity*rad_y/(radial_coord + devx) 
        tan_velocity = np.sqrt(tvx**2 + tvy**2)
        temperatures = Temp_S(1, internal_energy)

        edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) & (radius <= 30)#(radius <= center_boxsize/2*np.sqrt(3))

        # stat, x_edge, z_edge = make_voronoi_slice_edge(coord[edge_mask], density[edge_mask], n_bins, half_boxsize, 40)
        # X, Z = np.meshgrid(x_edge,z_edge)  
        
        plot_edge(axs[x], coord[edge_mask], number_density[edge_mask], n_bins, half_boxsize, center_boxsize, 1e-5, 1, log=True)

        density_mesh = axs[x].collections[0]
        density_mesh.set_cmap("mako")
        # if x != 0: plt.setp(axs[i].get_xticklabels()[0], visible=False)    

        axs[x].text(0.02, 0.93,'t =' + str(ts) + " Myr" , transform=axs[x].transAxes, fontsize=14, color="white")
        # axs[x].text(0.02, 0.93,'t =' + str(ts) + " Myr" , transform=axs[x].transAxes, fontsize="medium", color="white")
        axs[x].set_xticks(ticks)
        axs[x].set_yticks(ticks)

        axs[x].set(xlabel='', ylabel='')
        axs[x].tick_params(axis='both', which='major', labelsize=12)
        if x % 4 == 0:
            print(E_load)
            print(M_load)
            axs[x].text(0.02, 0.87, labeling[i] , transform=axs[x].transAxes, fontsize=14, color="white")
            axs[x].text(0.02, 0.80, r"$\alpha = \beta = 0.25$, $R_{inject}=$" + str(R) + " kpc", transform=axs[x].transAxes, fontsize=14, color="white")
            # axs[x].text(0.02, 0.72,r"$R_{inject}=$" + str(R) + " kpc", transform=axs[x].transAxes, color="white", fontsize="large")
            # background_rect = patches.Rectangle((0, 0.7), width=1, height=0.3, color='black', alpha=0.25, transform=axs[x].transAxes, fill=True)
            # axs[x].add_patch(background_rect)
            # plt.setp(axs[x].get_xticklabels()[-1], visible=False)

        if x not in [3, 7, 11]:
            plt.setp(axs[x].get_xticklabels()[-1], visible=False)
        if x not in [0,4,8]:
            axs[x].set_yticklabels([])
        x += 1


fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.85, 0.125, 0.018, 0.755])

cbar_masses = fig.colorbar(density_mesh, cax=cbar_ax)
labels_cbar = [1e-5*(10**(x)) for x in range(0,6)]
cbar_masses.set_ticks(labels_cbar)
cbar_masses.set_ticklabels([round(np.log10(label)) for label in (labels_cbar)])
cbar_masses.set_label(r"Density  [$\rm log_{10}(n)$]", fontsize=14)
cbar_masses.ax.tick_params(labelsize=14)

fig.text(0.5, 0.08, 'X [kpc]', ha='center', fontsize=14)
fig.text(0.09, 0.5, 'Z [kpc]', va='center', rotation='vertical', fontsize=14)

fig.subplots_adjust(wspace = 0)

plt.savefig("density_disk_comparison_edge.pdf", bbox_inches='tight', dpi=150)
plt.show()


## Time Evolution of Gas Temperatures

In [ ]:

# Non-Cooling
## Injection Radius
# outputs = ["./cc85_disk/output_center_ref/", "output_r300oc"]
## Disk Models
import matplotlib.patches as patches
outputs = ["./outflow_cooling/output_PIE_fid/snap_000.hdf5", "./outflow_cooling/output_PIE_fid/snap_005.hdf5", "./outflow_cooling/output_PIE_fid/snap_010.hdf5",
            "./outflow_cooling/output_PIE_fid/snap_015.hdf5",  "./outflow_cooling/output_PIE_fid/snap_020.hdf5",  "./outflow_cooling/output_PIE_fid/snap_025.hdf5"]

plt.style.use("seaborn-paper")


deviation = 5
histb_l = boxsize/2 - deviation  # boundary of histogram - lower bound
histb_h = boxsize/2  + deviation # boundary of histogram - upper bound


dx = center_boxsize/cells_per_dim
halfbox_inner = center_boxsize/2 
lower_bound, upper_bound = half_boxsize - dx*3, half_boxsize + dx*3
devx = dx/1e4

# labeling = ["M82/LMC", "MW", "SMC"]
###############################################################
######### SIMULATION DATA #########
data = {}
# snaps = times*2 # very roughly, I am taking a snapshot every 0.5 Myrs and since I have 100 snapshots, the snapshot with the closest time will be t*2
# snaps_alt = times
# ticks = [30, 35, 40, 45, 50, 55, 60, 65, 70]
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.set_rasterized(True)
# x = 0
n_bins = 300
# cmap = plt.cm.snsmako 
axs = axes.flatten()
for i, output in enumerate(outputs):
    with h5py.File(output,'r') as f:
        for key in f['PartType0']:
            data[key] = f['PartType0'][key][()]
        header = dict(f['Header'].attrs)
        parameters = dict(f['Parameters'].attrs)  
    times = header["Time"]    
    R = parameters["injection_radius"]
    M_load = parameters["M_load"]
    E_load = parameters["E_load"]
    coord = data["Coordinates"]
    x_coord = data["Coordinates"][:,0] 
    y_coord = data["Coordinates"][:,1]
    z_coord = data["Coordinates"][:,2]
    density = data["Density"]
    number_density = density*UnitNumberDensity
    internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
    vel_x = data["Velocities"][:,0]
    vel_y = data["Velocities"][:,1] 
    vel_z = data["Velocities"][:,2] 
    t = header["Time"]
    ts = np.round(t*1000, decimals=1)
    ''' Get the radius of the box - will be useful in later plots'''
    rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
    radial_coord = np.sqrt(rad_x**2+rad_y**2)
    radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)
    radial_velocity = (vel_x*rad_x + vel_y*rad_y)/(radial_coord + devx) 
    v_r_spherical = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + devx) 
    tvx, tvy = vel_x - radial_velocity*rad_x/(radial_coord + devx) , vel_y - radial_velocity*rad_y/(radial_coord + devx) 
    tan_velocity = np.sqrt(tvx**2 + tvy**2)
    abundance = data["ElectronAbundance"]
    temperatures = Temp_S(abundance, internal_energy)
    log10temp = np.log10(temperatures)
    edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) & (radius <= 10)#(radius <= center_boxsize/2*np.sqrt(3))

    plot_edge(axs[i], coord[edge_mask], log10temp[edge_mask], n_bins, half_boxsize, int(center_boxsize/2), 3, 7, log=False)
    temperature_mesh = axs[i].collections[0]
    temperature_mesh.set_cmap("plasma")
    if i not in [0, 3]: axs[i].set_ylabel('')
    if i <= 2: axs[i].set_xlabel('')

    if i == 0:
        background_rect = patches.Rectangle((0, 0.77), width=1, height=0.23, color='black', alpha=0.25, transform=axs[i].transAxes, fill=True)
        axs[i].add_patch(background_rect)
        axs[i].text(0.01, 0.95,"t = %0.1f Myr" % ts, transform=axs[i].transAxes, color="white", fontsize=14)
        axs[i].text(0.01, 0.90,r'LMC/M82 Disk - CIE+PIE', transform=axs[i].transAxes, color="white", fontsize=14)
        axs[i].text(0.020, 0.85,r"- $\beta = $" + str(M_load) +  r", $\alpha =$" + str(E_load) + r", $Z_{disk}= 4 Z_\odot$", transform=axs[i].transAxes, color="white", fontsize=14)
        axs[i].text(0.020, 0.78,r"- $\dot{M}_{SFR} =  20 \, M_\odot \, yr^{-1}$, " + r" $R_{inject} =$ " + str(int(R*1000)) + r"pc", transform=axs[i].transAxes, color="white", fontsize=14)
    else: axs[i].text(0.02, 0.94,"t = %0.1f Myr" % ts, transform=axs[i].transAxes, color="white", bbox=dict(facecolor='black', alpha=0.25), fontsize=14)
    axs[i].tick_params(axis='both', which='major', labelsize=12)


fig.subplots_adjust(right=0.84, hspace=0.1, wspace=0.1)
cbar_ax = fig.add_axes([0.85, 0.125, 0.018, 0.755])
cbar_T = fig.colorbar(temperature_mesh, cax=cbar_ax)
labels_cbar = [3, 4, 5, 6, 7]
cbar_T.set_ticks(labels_cbar)
cbar_T.set_label(r"Temperature  [${\rm log_{10}(K)}$]", fontsize=14)
cbar_T.ax.tick_params(labelsize=14)

plt.savefig("temperature_cooling_comparison_edge_shredding.pdf", bbox_inches='tight', dpi=300)
plt.show()


# The evolution of starbursts on periodic timescales 

In [ ]:
import matplotlib.patches as patches

# Non-Cooling
## Injection Radius
# outputs = ["./cc85_disk/output_center_ref/", "output_r300oc"]


# plt.style.use("seaborn-paper")
##### FOLDERS TO ANALYZE ######
outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_long_bursts/", "./outflow_cooling/output_short_bursts/", "./outflow_cooling/output_short_fast_bursts/"]
labeling = ["Continious", r"$\rm t_{burst} = 5 \, Myr,\,\, t_{rest} =  10 \, Myr$", 
          r"$\rm t_{burst} = 1 \, Myr,\,\, t_{rest} =  5 \, Myr$", r"$\rm t_{burst} = 1 \, Myr,\,\, t_{rest} =  1 \, Myr$"]
##############################

plt.style.use("seaborn-paper")

dx = center_boxsize/cells_per_dim
halfbox_inner = center_boxsize/2 
lower_bound, upper_bound = half_boxsize - dx*3, half_boxsize + dx*3
devx = dx/1e4

# labeling = ["Continious", "MW", "SMC"]
###############################################################
######### SIMULATION DATA #########
data = {}
times = np.array([10, 35, 50])
snaps = times*2 # very roughly, I am taking a snapshot every 0.5 Myrs and since I have 100 snapshots, the snapshot with the closest time will be t*2
snaps_alt = times
ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90] # 20, 30, 40, 50, 60, 70, 80
fig, axes = plt.subplots(4, 3, figsize=(13, 16))
fig.set_rasterized(True)
x = 0
n_bins = 300
# cmap = plt.cm.snsmako 
axs = axes.flatten()
for i, output in enumerate(outputs):
    s = snaps_alt if any(o in output for o in ["large", "small"]) else snaps
    for j, snap in enumerate(s):
        filename = output + "snap_%03d.hdf5" % snap
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)      
        R = parameters["injection_radius"]
        M_load = parameters["M_load"]
        E_load = parameters["E_load"]
        coord = data["Coordinates"]
        x_coord = data["Coordinates"][:,0] 
        y_coord = data["Coordinates"][:,1]
        z_coord = data["Coordinates"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity
        internal_energy = data["InternalEnergy"] # NOTE: This is specific internal energy, not the actual internal energy
        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1] 
        vel_z = data["Velocities"][:,2] 
        t = header["Time"]
        ts = int(round(t*1000))
        ''' Get the radius of the box - will be useful in later plots'''
        rad_x, rad_y, rad_z = x_coord - 0.5*boxsize, y_coord - 0.5*boxsize, z_coord - 0.5*boxsize
        radial_coord = np.sqrt(rad_x**2+rad_y**2)
        radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)
        radial_velocity = (vel_x*rad_x + vel_y*rad_y)/(radial_coord + devx) 
        v_r_spherical = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + devx) 
        tvx, tvy = vel_x - radial_velocity*rad_x/(radial_coord + devx) , vel_y - radial_velocity*rad_y/(radial_coord + devx) 
        tan_velocity = np.sqrt(tvx**2 + tvy**2)
        x_e = data["ElectronAbundance"]

        temperatures = Temp_S(x_e, internal_energy)

        edge_mask = (y_coord >=lower_bound) & (y_coord <= upper_bound) & (radius <= 50)#(radius <= center_boxsize/2*np.sqrt(3))

        plot_edge(axs[x], coord[edge_mask], temperatures[edge_mask], n_bins, half_boxsize, boxsize, 1e3, 1e8, log=True)

        temperature_mesh = axs[x].collections[0]
        temperature_mesh.set_cmap("inferno")
        # if x != 0: plt.setp(axs[i].get_xticklabels()[0], visible=False)    

        axs[x].text(0.02, 0.93,'t =' + str(ts) + " Myr" , transform=axs[x].transAxes, fontsize=14, color="white")
        # axs[x].text(0.02, 0.93,'t =' + str(ts) + " Myr" , transform=axs[x].transAxes, fontsize="medium", color="white")


        axs[x].tick_params(axis='both', which='major', labelsize=13)
        axs[x].tick_params(axis='both', which='minor', labelsize=13)
        axs[x].set(xlabel='', ylabel='')

        if x % 3 == 0:
            print(E_load)
            print(M_load)
            # axs[x].text(0.02, 0.87, labeling[i] , transform=axs[x].transAxes, fontsize="large", color="white")
            axs[x].text(0.02, 0.845, labeling[i],  transform=axs[x].transAxes, fontsize=14, color="white") #  bbox=dict(facecolor='white', edgecolor='gray'))
            # axs[x].text(0.02, 0.72,r"$R_{inject}=$" + str(R) + " kpc", transform=axs[x].transAxes, color="white", fontsize="large")
            # background_rect = patches.Rectangle((0, 0.7), width=1, height=0.3, color='black', alpha=0.25, transform=axs[x].transAxes, fill=True)
            # axs[x].add_patch(background_rect)
            # plt.setp(axs[x].get_xticklabels()[-1], visible=False)

        background_rect = patches.Rectangle((0, 0.82), width=1, height=0.18, color='black', alpha=0.25, transform=axs[x].transAxes, fill=True)
        axs[x].add_patch(background_rect)
        
        if x < 9:
            axs[x].set_xticklabels([])
        else: axs[x].set_xticks(ticks)

        if x == 10:
            plt.setp(axs[x].get_xticklabels()[-1], visible=False)
            plt.setp(axs[x].get_xticklabels()[0], visible=False)

        if x not in [0, 3, 6, 9]:
            axs[x].set_yticklabels([])
        else: axs[x].set_yticks(ticks)
        if x == 3 or x == 6:
            print(axs[x].get_yticklabels())
            plt.setp(axs[x].get_yticklabels()[0], visible=False)
        if x == 3: 
            plt.setp(axs[x].get_yticklabels()[-1], visible=False)
        x += 1



fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.84, 0.125, 0.018, 0.755])

fig.text(0.5, 0.08, 'X [kpc]', ha='center', fontsize=14)
# if FACE_ON: fig.text(0.1, 0.5, 'Y [kpc]', va='center', rotation='vertical')
fig.text(0.070, 0.5, 'Z [kpc]', va='center', rotation='vertical', fontsize=14)

fig.subplots_adjust(wspace = 0, hspace= 0.00)

cbar_T = fig.colorbar(temperature_mesh, cax=cbar_ax, pad=0.00)
labels_cbar = [1e3*(10**(x)) for x in range(0,6)]
cbar_T.set_ticks(labels_cbar)
cbar_T.ax.tick_params(labelsize=13)
cbar_T.set_ticklabels([round(np.log10(label)) for label in (labels_cbar)])
cbar_T.set_label(r"Temperature  [$\rm log_{10}(K)$]", fontsize=14)


plt.savefig("duty_T_time_comparison.pdf", bbox_inches='tight', dpi=300)
plt.show()